<a href="https://colab.research.google.com/github/Prasannnnn/Real-Time-Share-Price-Scraping-and-Analysis/blob/main/Real_Time_Share_Price_Scraping_and_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
def scrape_screener(stock_name):
    """Scrapes financial data of a given stock from Screener.in"""
    url = f"https://www.screener.in/company/{stock_name}/"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                      'Chrome/84.0.4147.105 Safari/537.36'
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        return {"Error": f"Failed to fetch data for {stock_name}. Check stock symbol."}

    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        company_name = extract_company_name(soup)

        # Extract financial metrics
        market_cap = extract_value(soup, 'Market Cap')
        current_price = extract_value(soup, 'Current Price')
        high_low = extract_value(soup, 'High / Low')
        stock_pe = extract_value(soup, 'Stock P/E')
        dividend_yield = extract_value(soup, 'Dividend Yield')
        roce = extract_value(soup, 'ROCE')
        roe = extract_value(soup, 'ROE')

        # Perform Sentiment Analysis
        sentiment = get_sentiment(stock_pe, roce, roe, current_price, high_low, dividend_yield)

        data = {
            'Company Name': company_name,
            'Market Cap': market_cap,
            'Current Price': current_price,
            'High / Low': high_low,
            'Stock P/E': stock_pe,
            'Dividend Yield': dividend_yield,
            'ROCE': roce,
            'ROE': roe,
            'Recommendation': sentiment
        }

        df = pd.DataFrame([data])
        return df

    except Exception as e:
        return {"Error": f"Parsing error: {e}"}


In [23]:
def extract_company_name(soup):
    """Extracts the company name from the page."""
    company_name_tag = soup.find('h1', class_="company-name") or soup.find('h1', class_='margin-0')
    return company_name_tag.text.strip() if company_name_tag else 'N/A'

In [24]:

def extract_value(soup, label):
    """Extracts financial values based on the label."""
    try:
        label_tag = soup.find(string=lambda text: text and label in text)
        if label_tag:
            parent = label_tag.parent
            value_tag = parent.find_next(class_='number')
            return value_tag.text.strip() if value_tag else 'N/A'
        return 'N/A'
    except Exception:
        return 'N/A'

In [25]:
def get_sentiment(stock_pe, roce, roe, current_price, high_low, dividend_yield):
    """Performs basic sentiment analysis to decide Buy, Hold, or Sell"""

    try:
        pe_ratio = float(stock_pe) if stock_pe.replace('.', '', 1).isdigit() else None
        roce_value = float(roce.replace('%', '')) if roce.replace('%', '').replace('.', '', 1).isdigit() else None
        roe_value = float(roe.replace('%', '')) if roe.replace('%', '').replace('.', '', 1).isdigit() else None
        div_yield = float(dividend_yield.replace('%', '')) if dividend_yield.replace('%', '').replace('.', '', 1).isdigit() else None

        # Extract high and low values from High/Low string
        if "/" in high_low:
            low, high = map(lambda x: float(x.strip()), high_low.split('/'))
            price_ratio = (float(current_price) - low) / (high - low)
        else:
            price_ratio = None

        # Decision Logic
        if pe_ratio and pe_ratio < 20 and roce_value and roce_value > 15 and roe_value and roe_value > 15:
            return "BUY 📈 (Good Fundamentals)"
        elif price_ratio and price_ratio > 0.8:  # If price is close to 52-week high
            return "WAIT ⏳ (Too Expensive)"
        elif price_ratio and price_ratio < 0.2:  # If price is near 52-week low
            return "BUY 📉 (Undervalued)"
        elif div_yield and div_yield > 4:  # High dividend yield
            return "BUY 💰 (Good for Dividends)"
        elif pe_ratio and pe_ratio > 40:  # High P/E ratio
            return "SELL ❌ (Overvalued)"
        else:
            return "HOLD 🤔 (Needs Further Analysis)"

    except Exception:
        return "HOLD 🤔 (Insufficient Data)"

In [26]:
# **User Input for Stock Name**
stock_name = input("Enter stock symbol (e.g., TCS, RELIANCE, INFY): ").upper()

# **Run the scraper**
result = scrape_screener(stock_name)

# **Display result**
if isinstance(result, pd.DataFrame):
    print(result)
else:
    print(result["Error"])

Enter stock symbol (e.g., TCS, RELIANCE, INFY): tcs
                    Company Name Market Cap Current Price High / Low  \
0  Tata Consultancy Services Ltd  14,74,921         4,073      4,592   

  Stock P/E Dividend Yield  ROCE   ROE                   Recommendation  
0      30.5           1.35  74.8  60.4  HOLD 🤔 (Needs Further Analysis)  
